In [ ]:
%%capture
!pip install scanpy
!pip install KDEpy
!pip install leidenalg

In [ ]:
%%capture
!pip install -e ../../tools/nomad/

In [13]:
import os
import sys
from datetime import datetime

import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
from scipy.stats import spearmanr

# to be able to import from tools dir
module_path = os.path.abspath(os.path.join("../../"))
if module_path not in sys.path:
    sys.path.append(module_path)

import tools.util_probe as up
import tools.util as ut
import tools.NB_est as nb
import tools.countsplit as cs
import tools.scDEED as scd
import tools.clustering_opt as co
import tools.util_plot as nmd_plot
import tools.util_imputation as nmd_imp

import fi_nomad as nmd
from fi_nomad.types import kernelInputTypes
from fi_nomad.types import KernelStrategy
from fi_nomad.types import InitializationStrategy

import warnings

warnings.filterwarnings("ignore")

from scipy.stats import kendalltau
import importlib
import logging

In [14]:
# logging has to manually turned on to see nomad output
logging.basicConfig(level=logging.INFO)

## Read data

In [15]:
data_path = "../../data/S3"
figure_path = f"{data_path}/figures/imputation"
layer = "counts"

In [16]:
data_counts = sc.read_h5ad(data_path + "/filtered_data_maxpool_processed_and_pca.h5ad")
data_counts

AnnData object with n_obs × n_vars = 1544 × 5553
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'outlier', 'n_genes', 'total_counts_norm', 'total_counts_scale', 'embedding_reliability', 'reliability_score', 'null_reliability_score', 'leiden_opt_PCA'
    var: 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'var_counts', 'is_scd_outlier', 'nb_overdisp', 'nb_overdisp_cutoff', 'nb_mean', 'nb_umi', 'Intercept_step1_sct', 'log_umi_step1_sct', 'dispersion_step1_sct', 'mean', 'std'
    uns: 'BacSC_params_PCA', 'PCA', 'leiden', 'leiden_opt_PCA_colors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'normalized_counts', 'vst_counts'
    obsp: 'PCA_connectivities', 'PCA_distances'

Read expression level class data

In [17]:
expr_class = pd.read_csv(
    "../../data/control_expression_classification.tsv", delimiter="\t"
)
expr_class.set_index(keys="locus_tag", inplace=True)
expr_class

,Name,tpm,expression_level,log_tpm
locus_tag,,,,
PA0001,dnaA,333.097801,high,8.379802
PA0002,dnaN,299.127321,high,8.224616
PA0003,recF,128.151975,medium_high,7.001712
PA0004,gyrB,138.579052,medium_high,7.114565
PA0005,lptA,141.413217,medium_high,7.143773
...,...,...,...,...
PA5566,PA5566,45.077831,medium_low,5.494346
PA5567,PA5567,49.149167,medium_low,5.619095
PA5568,PA5568,208.178186,high,7.701675


- `mean_counts`: mean gene expression
- `n_cells`: number of cells that express gene (so (n - `n_cells`) / n = sparsity)

In [18]:
data_counts.var = data_counts.var.join(expr_class)
data_counts.var

,feature_types,genome,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,var_counts,...,nb_umi,Intercept_step1_sct,log_umi_step1_sct,dispersion_step1_sct,mean,std,Name,tpm,expression_level,log_tpm
PA0001,Gene Expression,PA01,735,0.666451,0.510696,52.396373,1029.0,6.937314,735,0.755973,...,2.182498,-6.941972,2.135988,0.009296,1.608654,0.117757,dnaA,333.097801,high,8.379802
PA0002,Gene Expression,PA01,541,0.468912,0.384522,64.961140,724.0,6.586172,541,0.588412,...,2.249176,NaN,NaN,NaN,1.229464,0.142842,dnaN,299.127321,high,8.224616
PA0003,Gene Expression,PA01,890,0.893782,0.638576,42.357513,1380.0,7.230563,890,1.043122,...,2.153516,NaN,NaN,NaN,1.691644,0.124664,recF,128.151975,medium_high,7.001712
PA0004,Gene Expression,PA01,463,0.373705,0.317511,70.012953,577.0,6.359574,463,0.398557,...,2.293357,NaN,NaN,NaN,0.974741,0.150374,gyrB,138.579052,medium_high,7.114565
PA0005,Gene Expression,PA01,173,0.120466,0.113745,88.795337,186.0,5.231109,173,0.124089,...,2.494667,NaN,NaN,NaN,-0.471332,0.257211,lptA,141.413217,medium_high,7.143773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PA5566,Gene Expression,PA01,14,0.009715,0.009668,99.093264,15.0,2.772589,14,0.010916,...,2.336949,NaN,NaN,NaN,-3.570001,0.321510,PA5566,45.077831,medium_low,5.494346
PA5567,Gene Expression,PA01,163,0.126943,0.119509,89.443005,196.0,5.283204,163,0.161347,...,2.493876,NaN,NaN,NaN,-0.463125,0.270051,PA5567,49.149167,medium_low,5.619095
PA5568,Gene Expression,PA01,740,0.659974,0.506802,52.072539,1019.0,6.927558,740,0.750315,...,2.183110,NaN,NaN,NaN,1.604845,0.117774,PA5568,208.178186,high,7.701675
PA5569,Gene Expression,PA01,1430,3.498057,1.503646,7.383420,5401.0,8.594525,1430,14.170981,...,2.223133,-6.188664,2.421585,0.057214,1.302429,0.374002,rnpA,3387.343294,high,11.725938


## Fit NMD

In [19]:
latent_rank = 10
momentum_beta = 0.7
max_iterations = 1000

init_strat = InitializationStrategy.ROWWISE_MEAN
kernel_strat = KernelStrategy.MOMENTUM_3_BLOCK_MODEL_FREE

In [20]:
m, n = data_counts.X.shape
X_nmd = ut.convert_to_dense(data_counts, layer=layer)
X_nmd = X_nmd.astype(float)

In [21]:
result = nmd.decompose(
    X_nmd,
    latent_rank,
    kernel_strategy=kernel_strat,
    initialization=init_strat,
    kernel_params=kernelInputTypes.Momentum3BlockAdditionalParameters(
        momentum_beta=momentum_beta
    ),
    manual_max_iterations=max_iterations,
    verbose=True,
)

INFO:fi_nomad.entry:	Initiating run, target_rank: 10, tolerance: None
INFO:fi_nomad.entry:1000 total, final loss Not Tracked
INFO:fi_nomad.entry:	Initialization took 32.63490094896406 loop took 230.56548640300753 overall (0.23056548640300753/ea)


In [22]:
data_counts.obsm["X_nmd"] = result.factors[0]
data_counts.varm["NMD_components"] = result.factors[1].T
data_counts.layers["Theta"] = result.factors[0] @ result.factors[1]

Output file for pretty plotting in R

In [24]:
pd.DataFrame(data_counts.layers["Theta"], columns=data_counts.var_names).to_csv(
    f"{data_path}/Theta_3BNMD_r{latent_rank}.csv", index=False
)

## Sample gene pairs

In [37]:
importlib.reload(nmd_imp)

<module 'tools.util_imputation' from '/Users/stffn/projects/thesis_dev/msc_thesis/application/tools/util_imputation.py'>

In [38]:
n_genes = 50  # number of gene pairs
expression_levels_2_compare = ["low", "medium_high"]

In [39]:
gene_pairs = nmd_imp.sample_gene_pairs(
    data_counts,
    n=n_genes,
    measure_var=["mean_counts", "pct_dropout_by_counts"],
    expr_lvls=expression_levels_2_compare,
)
gene_pairs.head()

,low,medium_high,dist,gene_pair_id
0,PA0688,PA0109,0.000000,0
1,PA2908,PA4791,0.000012,1
2,PA4293,PA0176,0.000012,2
3,PA3760,PA0108,0.000104,3
4,PA1227,PA3205,0.000012,4


Get all negative Theta values.

In [40]:
gene_pair_df = nmd_imp.make_gene_pair_df(gene_pairs, adata=data_counts)
gene_pair_df.sort_values(by=["gene_pair_id", "dist"]).head()

,Gene,Expression,gene_pair_id,dist,expression_lvl_class
0,PA0688,0.005996,0,0.0,low
1,PA0688,0.011742,0,0.0,low
2,PA0688,0.010352,0,0.0,low
3,PA0688,0.022282,0,0.0,low
4,PA0688,0.019896,0,0.0,low


## Export to csv for visualization in R

In [41]:
gene_pair_df.to_csv(
    f"{data_path}/gene_pair_df_4_viz_{n_genes}genes_3BNMD_rank{latent_rank}_{expression_levels_2_compare[0]}-{expression_levels_2_compare[1]}_mean_and_sparsity_whereXzero.csv",
    index=False,
)